<a href="https://colab.research.google.com/github/gusdn1503/-/blob/master/FEST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
10.127471159999999
GPU (s):
1.8042616820000035
GPU speedup over CPU: 5x


In [0]:
import tensorflow as tf
import glob 
import os
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd drive 

/content/drive


In [7]:
cd My Drive

/content/drive/My Drive


In [8]:
cd data

/content/drive/My Drive/data


In [9]:
ls -ltr

total 8
drwx------ 2 root root 4096 Jan 13 13:44 FEST_train/
drwx------ 2 root root 4096 Jan 13 14:24 FEST_test/


In [10]:
cd FEST_test

/content/drive/My Drive/data/FEST_test


In [11]:
test_filenames=glob.glob('*.jpg')
len(test_filenames)

2032

In [12]:
cd ..

/content/drive/My Drive/data


In [13]:
cd FEST_train

/content/drive/My Drive/data/FEST_train


In [14]:
train_filenames=glob.glob('*.jpg')
len(train_filenames)

8586

In [15]:
def label_getter(raw_filenames):
    
    label=[]
    for i in raw_filenames:
        a=i.split('\\')[-1]
        b=a.split('_')[0]
        label.append(b)
    return label




train_labels=label_getter(train_filenames)

test_labels=label_getter(test_filenames)
print(test_labels)

['10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10047545284', '10100214433', '10100214433', '10100214433', '10100214433', '10100214433', '10100214433', '10100214433', '10100214433', '10100214433', '10100214433', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471732649', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471782012', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10471787412', '10476902449', '10476902

In [0]:





def make_int_label(labels):
    i_label=[]
    count=0
    a='10047545284'

    for label in train_labels:
        if label==a:
            i_label.append(count)
        if label!=a:
            count+=1
            a=label
            i_label.append(count)
    return i_label

train_labels=make_int_label(train_labels)
test_labels=make_int_label(test_labels)


dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
# or dataset = tf.data.TFRecordDataset(filenames)
#dataset = tf.data.Dataset.zip((filename,label))
dataset = dataset.shuffle(buffer_size=10000)


def parse_function(filename, label):
    image_string = tf.read_file(filename)
    label=tf.cast(label,tf.int32)
    one_hot = tf.one_hot(label, 153) #Num_classes=153
    label=tf.reshape(one_hot,[-1,153])
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1] or image = tf.cast(image, tf.float32)
    image = tf.image.convert_image_dtype(image, tf.float32)

    #image = tf.image.resize_images(image, [64, 64])
    return image, label


dataset = dataset.map(parse_function, num_parallel_calls=4)
#dataset = dataset.apply(tf.contrib.data.unbatch())
#dataset = dataset.batch(32)
dataset = dataset.padded_batch(32, padded_shapes=([300, 300, 3], [None,153]))
dataset = dataset.repeat()
dataset= dataset.prefetch(1)
iterator = dataset.make_one_shot_iterator()

x,y = iterator.get_next()

In [0]:
def conv_net(x, n_classes, keep_prob):
# Define a scope for reusing the variables
#with tf.variable_scope('ConvNet'):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    X= tf.reshape(x, shape=[-1, 300, 300, 3])


    #W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01)
    #L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
   # L1 = tf.nn.relu(L1)
   # L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

    #or high level API : Convolution Layer with 32 filters and a kernel size of 5
    conv1 = tf.layers.conv2d(X, 32, 3, activation=tf.nn.relu)
    # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
    #L1 = tf.layers.dropout(conv1, rate=keep_prob)
    conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
    # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    '''
    # L2 ImgIn shape=(?, 150, 150, 32)
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
    #    Conv      ->(?, 150, 150, 64)
    #    Pool      ->(?, 75, 75, 64)
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.layers.dropout(L2, rate=keep_prob)
    # L3 ImgIn shape=(?, 75, 75, 64)
    W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
    #    Conv      ->(?, 75, 75, 128)
    #    Pool      ->(?, 25, 25, 128)
    #    Reshape   ->(?, 25* 25* 128) # Flatten them for FC
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 3, 3, 1], strides=[1, 3, 3, 1], padding='SAME')
    L3 = tf.layers.dropout(L3, rate=keep_prob)
    L3_flat = tf.reshape(L3, [-1, 128 * 25 * 25])
    L3_flat=tf.cast(L3_flat,tf.float32)
    print("ok")
    '''
    # L4 FC 4x4x128 inputs -> 625 outputs
    #W4 = tf.get_variable("W4", shape=[128 * 25 * 25, 625],initializer=tf.contrib.layers.xavier_initializer())
    #b4 = tf.Variable(tf.random_normal([625]))
    #L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
    #L4 = tf.layers.dropout(L4, rate=keep_prob)
    
    fc1 = tf.contrib.layers.flatten(conv2)
    fc1 = tf.layers.dense(fc1, 1024)
    fc1 = tf.layers.dropout(fc1, rate=keep_prob)
    # L5 Final FC 625 inputs -> 10 output
    #W5 = tf.get_variable("W5", shape=[625, 153],initializer=tf.contrib.layers.xavier_initializer())
    #b5 = tf.Variable(tf.random_normal([153]))
    out = tf.layers.dense(fc1, 153) 
    return out
    '''
    # Convolution Layer with 32 filters and a kernel size of 5
    conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
    # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

    # Flatten the data to a 1-D vector for the fully connected layer
    fc1 = tf.contrib.layers.flatten(conv2)

    # Fully connected layer (in contrib folder for now)

    # Apply Dropout (if is_training is False, dropout is not applied)


    # Output layer, class prediction
    out = tf.layers.dense(fc1, n_classes)
    # Because 'softmax_cross_entropy_with_logits' already apply softmax,
    # we only apply softmax to testing network
    out = tf.nn.softmax(out) if not is_training else out

return out
'''

In [0]:
from tqdm import tnrange
from time import sleep


In [0]:
#tf.reset_default_graph()
logits_train = conv_net(x, 153, 0.8)
# Create another graph for testing that reuse the same weights, but has
# different behavior for 'dropout' (not applied).
logits_test = conv_net(x, 153, 1)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits_train, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
num_steps=2000
for step in tnrange(num_steps + 1):

    # Run optimization
    sess.run(train_op)

    if step % 200 == 0 or step == 1:
        # Calculate batch loss and accuracy
        # (note that this consume a new batch of data)
        loss, acc = sess.run([loss_op])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
